## Import modules

Set your user name as a case, pointing to your path to documents and tinyir.jar

In [ ]:
// set your case once
val (doc_dir: String, files_path: String) = System.getProperties().get("user.name").toString match {
    case "Yarden-"  => ("../documents", "../")
    case "Max"  => ("../MAXPATH", "../MAXPATH/")
}

In [ ]:
classpath.addPath(files_path + "tinyir-1.1.jar")

In [ ]:
import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

In [ ]:
import scala.io.Source  // for importing txt files
import java.io._  // for saving txt files
// import scala.collection.mutable.HashMap  //HashMap used for counting elements in linear time

In [ ]:
// import scala.util.Random
import scala.collection.mutable.{Map => MutMap, HashMap => MutHashMap}
// enables "mutable lists"
// import scala.collection.mutable.ListBuffer  
import scala.collection.mutable.{Set => MutSet}

In [ ]:
val timeit = new util.StopWatch

## Define classes and functions

In [ ]:
def token_filter(text_body: String) = {
    processing.StopWords.filterOutSW(
        processing.Tokenizer.tokenize(text_body.
                                      replaceAll("\\P{L}+", " "))
    ).
    map(x => PorterStemmer.stem(x)).filter(_.trim.nonEmpty).toList
}

In [ ]:
class xml_doc (file_path: String) {
    def get_doc(): xml.Elem = {
        XML.loadFile(file_path: String)
    }    
    
    def text() = {
        (get_doc() \\ "DOC" \\ "TEXT").text
    }
    
    def head() = {
        (get_doc() \\ "DOC" \\ "HEAD").text
    }

    def id() = {
        (get_doc() \\ "DOC" \\ "DOCNO").text.trim
    }
    
    def tokens() = {
        token_filter(head() ++ text())
    }
    
    def hash_tokens() = {
        tokens().map(x => x.hashCode())
    }
}

In [ ]:
def list_docs (path: String) = {  // : Array[java.io.File]
        new java.io.File(path).listFiles.map(x => x.toString())
    }
val numPattern = "[0-9]+".r

In [ ]:
val token_hash = MutHashMap[String, Int]() // token -> hash

def create_hash_doc_subset(star_count: Int, end_count: Int,
                           file_list: Array[String],
                           token_hash_map: MutHashMap[String, Int] = token_hash) = {
    val id_htoken = MutHashMap[Int, List[Int]]() // forward index, docID to tokens
    val htoken_id = MutHashMap[Int, List[Int]]()  // inverse index, tokens to docID
    val id_name = MutHashMap[Int, String]()  // inverse index, tokens to docID
    val name_id = MutHashMap[String, Int]()  // inverse index, tokens to docID
    var counter = star_count
    while (counter < end_count){
        var cur_doc = new xml_doc(file_list(counter))
        // get token from XML, then hash, or create hashes "on the fly"
        var cur_htoken = cur_doc.tokens.map(x => token_hash_map.getOrElseUpdate(x, token_hash_map.size))
        id_htoken += counter -> cur_htoken
        
        // update the inverse mapping, from (hashed) tokens to docID
        cur_htoken.distinct.foreach(
            (token: Int) => htoken_id(token) = htoken_id.getOrElseUpdate(token, List[Int]()) ++ List(counter)
        )
        
        id_name(counter) = cur_doc.id
        name_id(cur_doc.id) = counter
        
        counter += 1
        if (counter % 100 == 0) println(s"iteration $counter")
    }
    (id_htoken, htoken_id, token_hash_map, id_name, name_id)
}

In [ ]:
def write_int_to_intList(data: MutHashMap[Int, List[Int]], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var values = data(elem).toList
//         if(values.length>0){
            bw.write(elem+" "+values.mkString(" "))
            bw.newLine
//         }    
    }   
    bw.close()
}

def write_int_string(data: MutHashMap[Int, String], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var values = data(elem).toList
        if(values.length>0){
            bw.write(elem+" "+values.mkString(""))
            bw.newLine
        }    
    }   
    bw.close()
}

def write_string_int(data: MutHashMap[String, Int], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        bw.write(elem+" "+data(elem).toString)
        bw.newLine
    }   
    bw.close()
}

def write_int_to_int(data: MutHashMap[Int, Int], filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var value = data(elem)
            bw.write(elem+" "+value)
            bw.newLine
    }   
    bw.close()
}

In [ ]:
def load_mutmap_int_intList(path: String, mutmap: MutHashMap[Int, List[Int]]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
        val line_split = line.split(" ", -1).filter(_.trim.length > 0)
        mutmap(line_split.head.toInt) = 
            line_split.tail.map(x => x.toInt).toList
    }
}

def load_mutmap_int_string(path: String, mutmap: MutHashMap[Int, String]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
        val line_split = line.split(" ") // .filter(_.trim.length > 0)
        mutmap(line_split.head.toInt) = 
            line_split.last
    }
}

def load_mutmap_string_int(path: String, mutmap: MutHashMap[String, Int]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
//         val line_split = line.split(" ", -1)
        val line_split = line.split(" ") // .filter(_.trim.length > 0)
        mutmap(line_split.head) = 
            line_split.last.toInt
    }
}

def load_mutmap_int_int(path: String, mutmap: MutHashMap[Int, Int]) = {
    val lines = Source.fromFile(path).getLines.toList
    for (line <- lines){
        val line_split = line.split(" ", -1).filter(_.trim.length > 0)
        mutmap(line_split.head.toInt) = 
            line_split.last.toInt
    }
}

In [ ]:
val mb = 1024*1024
val runtime = Runtime.getRuntime
def print_memory() = {
    println(s"Used Memory:  " + (runtime.totalMemory - runtime.freeMemory) / mb)
    println(s"Free Memory:  " + runtime.freeMemory / mb)
    println(s"Total Memory: " + runtime.totalMemory / mb)
    println(s"Max Memory:   " + runtime.maxMemory / mb)
}

In [ ]:
val train_list = list_docs(doc_dir)

In [ ]:
val PATH_id_htoken = files_path + "id_htoken.txt"
val PATH_htoken_id = files_path + "htoken_id.txt"
val PATH_id_name = files_path + "id_name.txt"
val PATH_name_id = files_path + "name_id.txt"
val PATH_token_hash = files_path + "token_hash.txt"

val PATH_prun_htoken_collectfreq = files_path + "prun_htoken_collectfreq.txt"
val PATH_prun_htoken_id = files_path + "prun_htoken_id.txt"
val PATH_prun_id_htoken = files_path + "prun_id_htoken.txt"

# Importing data files and creating maps
# # not run

In [ ]:
// time it
timeit.start

val (id_htoken, htoken_id, token_hash, 
     id_name, name_id) = create_hash_doc_subset(0, 100000, train_list)

In [ ]:
// time it
timeit.uptonow / 60.0
// 87.56585954758334 , in minutes with 6GB
// 67.88821773650001 , in minutes with 7GB

In [ ]:
print_memory()

// Used Memory:  3981
// Free Memory:  1814
// Total Memory: 5796
// Max Memory:   5796

## Save to file

In [ ]:
write_int_to_intList(id_htoken, PATH_id_htoken)

In [ ]:
write_int_to_intList(htoken_id, PATH_htoken_id)

In [ ]:
write_int_string(id_name, PATH_id_name)

In [ ]:
write_string_int(name_id, PATH_name_id)

In [ ]:
write_string_int(token_hash, PATH_token_hash)

## Load from file

In [ ]:
// time it
timeit.start

val id_htoken: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val htoken_id: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val id_name: MutHashMap[Int, String] = MutHashMap[Int, String]()
val token_hash: MutHashMap[String, Int] = MutHashMap[String, Int]()
val name_id: MutHashMap[String, Int] = MutHashMap[String, Int]()

In [ ]:
load_mutmap_int_intList(PATH_id_htoken, id_htoken)
load_mutmap_int_intList(PATH_htoken_id, htoken_id)
load_mutmap_int_string(PATH_id_name, id_name)
load_mutmap_string_int(PATH_token_hash, token_hash)
load_mutmap_string_int(PATH_name_id, name_id)

// confirm load successful
// test_load_mutmap_id_htoken == id_htoken
// test_load_mutmap_htoken_id == htoken_id
// test_load_mutmap_id_name == id_name
// test_load_mutmap_token_hash == token_hash
// test_load_mutmap_name_id == name_id

In [ ]:
// time it
timeit.uptonow / 60.0
// 1.4485827969833334 , in minutes
// 1.3195113773833334 , in minutes

In [ ]:
print_memory()

// Used Memory:  3468
// Free Memory:  649
// Total Memory: 4117
// Max Memory:   5461

## Prune vocabulary, collection and document frequencies

In [ ]:
// htoken_id.mapValues(v => v.length).size
// 1356183
// htoken_id.mapValues(v => v.length).filter(_._2 > 5 - 1).size
// 176866
// reduction factor of ~7.67

val prun_threshold = 5
val pruned_token_set = htoken_id.mapValues(v => v.length).
    filter(_._2 > prun_threshold - 1).keys.toSet

In [ ]:
// time it
timeit.start

val prun_htoken_collectfreq: MutHashMap[Int, Int] = 
    MutHashMap(
        id_htoken.flatMap{ case (k,v) => v.filter(pruned_token_set.contains(_)) }.
        groupBy(identity).mapValues(_.size)
        .toSeq:_*)

prun_htoken_collectfreq.size

timeit.uptonow / 60.0
// 7.0919255263  , in minutes
// 0.39257662956666667 , in minutes

In [ ]:
// time it
timeit.start

val prun_htoken_id: MutHashMap[Int, List[Int]] = 
    MutHashMap(
        htoken_id.filterKeys(
            pruned_token_set.contains(_)
        ).toSeq:_*)

prun_htoken_id.size

timeit.uptonow / 60.0
// 0.012546176999999999 , in minutes

In [ ]:
// time it
timeit.start

val prun_id_htoken: MutHashMap[Int, List[Int]] = 
    MutHashMap(
//         id_htoken.flatMap{ case (k,v) => (k, v.filter(pruned_token_set.contains(_))) }.
        id_htoken.mapValues{ v => v.filter(pruned_token_set.contains(_)) }.
        toSeq:_*)

prun_id_htoken.size

timeit.uptonow / 60.0
// 0.20076514550000002 , in minutes

## Save pruned results to file

In [ ]:
write_int_to_int(prun_htoken_collectfreq, PATH_prun_htoken_collectfreq)

In [ ]:
write_int_to_intList(prun_htoken_id, PATH_prun_htoken_id)

In [ ]:
write_int_to_intList(prun_id_htoken, PATH_prun_id_htoken)

## Load maps (pruned)
## # strat from here

In [ ]:
// time it
timeit.start

val prun_htoken_collectfreq: MutHashMap[Int, Int] = MutHashMap[Int, Int]()
val prun_id_htoken: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val prun_htoken_id: MutHashMap[Int, List[Int]] = MutHashMap[Int, List[Int]]()
val id_name: MutHashMap[Int, String] = MutHashMap[Int, String]()
val token_hash: MutHashMap[String, Int] = MutHashMap[String, Int]()
val name_id: MutHashMap[String, Int] = MutHashMap[String, Int]()

In [ ]:
load_mutmap_int_int(PATH_prun_htoken_collectfreq, prun_htoken_collectfreq)
load_mutmap_int_intList(PATH_prun_id_htoken, prun_id_htoken)
load_mutmap_int_intList(PATH_prun_htoken_id, prun_htoken_id)
load_mutmap_int_string(PATH_id_name, id_name)
load_mutmap_string_int(PATH_token_hash, token_hash)
load_mutmap_string_int(PATH_name_id, name_id)

In [ ]:
// time it
timeit.uptonow / 60.0
// 2.1530588158666664 , in minutes

In [ ]:
print_memory()

// Used Memory:  2569
// Free Memory:  1782
// Total Memory: 4352
// Max Memory:   5461

## Language model score

# Testing ground

In [ ]:
// Used Memory:  400
// Free Memory:  215
// Total Memory: 616
// Max Memory:   3641

In [ ]:
val token_hm = MutHashMap[String, Int]()
List("word1", "word3").map(x => token_hm.getOrElseUpdate(x, token_hm.size))

In [ ]:
classpath.addPath(tiny_path)

In [ ]:
trait Result[T] extends Any {
    def id : Int
    def matches(that: T) : Int                 
    def isMatch(that: T) = matches(that)==0
    def matched(that: T) : T    
}

object InvertedIndex {
    // generic list intersection (does not require sorted lists)
    private def unsortedIntersect [A<% Result[A]](l1: List[A], l2: List[A]) = l1.intersect(l2)

    // optimized list intersection for sorted posting lists 
    // uses "matches" and "matched" methods to work for all posting types
    def sIntersect[A <% Result[A]] (l1: List[A], l2: List[A]) : List[A] = {
        @annotation.tailrec
        def iter (l1: List[A], l2: List[A], result: List[A]) : List[A] = {
            if (l1.isEmpty || l2.isEmpty) 
                result.reverse
            else (l1.head matches l2.head) match {
                case n if n>0 => iter(l1, l2.tail,result)  // advance list l2
                case n if n<0 => iter(l1.tail, l2,result)  // advance list l1
                case _        => iter(l1.tail, l2.tail, (l1.head matched l2.head)::result)	      
            }
        }    
        iter(l1,l2,Nil)      
    }
}

abstract class InvertedIndex[Res <% Result[Res]]  {
    def results (term: String) : List[Res] 
    def results (terms: Seq[String]) : List[Res] = {
        val resultLists      = terms.map(term => results(term))
        val shortToLongLists = resultLists.sortWith( _.length < _.length) 
        shortToLongLists.reduceLeft( (l1,l2) => InvertedIndex.sIntersect(l1,l2) )
    }
}

// import ch.ethz.dal.tinyir.indexing.InvertedIndex

In [ ]:
import scala.math._

In [ ]:
class Document(val id: Int, val tokens: List[Int])
//     def id: Int = this.id
//     def tokens: List[Int] = this.tokens

In [ ]:
case class ProxResult(val id: Int, val lpos: Int, val rpos: Int) extends Result[ProxResult] {
    def matches(that: ProxResult) : Int = {    
        if (this.id != that.id) this.id - that.id
        else if ((max(rpos,that.rpos) - min(lpos,that.lpos)) <= ProxWindow.size) 0 // match
        else this.lpos-that.lpos  // advance in list with the minimal lpos
    }
    def matched(that: ProxResult) = 
        ProxResult(id, min(this.lpos,that.lpos), max(this.rpos,that.rpos))
}

object ProxWindow {
    var size = 1
    def setSize(w: Int) {assert(w>=1); size = w}
}

class PosIndex (docs: Stream[Document]) extends InvertedIndex[ProxResult] {

    case class PosPosting(val id: Int, val pos: Int) extends Ordered[PosPosting] {
        def this(t: PosTuple) = this(t.id, t.pos) 
//         def compare(that: PosPosting) = Ordering[Tuple2[Int, Int]].compare((this.id, this.pos), (that.id, that.pos) ) 
    }
    type PostList = List[PosPosting]
    val index : Map[String, PostList] = {
        val groupedPostings = postings(docs).groupBy(_.term)
        groupedPostings.mapValues(_.map(p => PosPosting(p.id,p.pos)).sorted)
    }
  
    case class PosTuple(term: String, id: Int, pos: Int) 
    def postings (s: Stream[Document]): List[PosTuple] =
        s.flatMap( d => d.tokens.zipWithIndex.map{ case (tk,pos) => PosTuple(tk,d.ID,pos) } ).toList

    override def results (word: String) : List[ProxResult] = 
        index.getOrElse(word,null).map(p => ProxResult(p.id, p.pos, p.pos))
    override def results (terms: Seq[String]) : List[ProxResult] = results(terms,1)
    def results (terms: Seq[String], win: Int) : List[ProxResult] = {
        val resultLists = terms.map(term => results(term))
        val shortToLongLists = resultLists.sortWith( _.length < _.length)   
        shortToLongLists.reduceLeft( (l1,l2) => InvertedIndex.sIntersect(l1,l2) )
    } 
}